In [ ]:
# Googleドライブをマウント
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Open3dをインストール
!pip install open3d

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 447.7/447.7 MB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 87.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.8/139.8 kB 9.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 56.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 41.4 MB/s eta 0:00:00
  Attempting uninstall: widgetsnbextension
    Found existing installation: widgetsnbextension 3.6.10
    Uninstalling widgetsnbextension-3.6.10:
      Successfully uninstalled widgetsnbextension-3.6.10
  Attempting uninstall: ipywidgets
    Found existing installation: ipywidgets 7.7.1
    Uninstalling ipywidgets-7.7.1:
      Successfully uninstalled ipywidgets-7.7.1


In [ ]:
# CityGML(XML)ファイルを読み込むためのモジュール
!pip install lxml
# 座標系の変換をするモジュール
!pip install pyproj
# 3次元表示Plotlyをインストール
!pip install plotly

In [9]:
# CityGML(XML)ファイルを準備
import os

# CityGML(XML)ファイル
filename = "/content/drive/MyDrive/13.Colab Notebooks/dataset/citygml_in/53392633_bldg_6697_2_op.gml"
# CityGML(XML)ファイルのファイル名のみを取得
basename = os.path.basename(filename)

# ファイル名を分割
basenames = basename.split('_')
# ファイル名からメッシュコードを取得
mesh_code = basenames[0]
# ファイル名から地物型 (bldg)
object_name = basenames[1]
# ファイル名からCRS 空間参照 ID (SRID)　(座標情報を取得)
from_srid = basenames[2]

# 出力するファイルの座標系を選択
# 座標系に関しては下記を参考にしてください
# はじめての地理空間情報: https://hackmd.io/5S_fL0GsT2OTkIi0owIccw

# 東京都大田区羽田空港のデータなので、東京都の座標・測地系で出力する
# 日本測地系2011　平面直角座標系: 東京都の一部、福島県、栃木県、茨城県、埼玉県、千葉県、群馬県、神祭川県
to_srid = "6677"

In [10]:
import lxml
from lxml import etree

# CityGML(XML)ファイルを読み込み
tree = etree.parse(filename)
root = tree.getroot()
# CityGML(XML)ファイルのスキーマ情報を取得
nsmap = root.nsmap

In [11]:
import numpy as np

# 文字列データから配列に変換
def str2floats(x):
    """ x y z -> [x, y, z] """
    return np.array([float(i) for i in x.text.split(' ')])

読み込んだ平面データを三角形のメッシュデータに変換するために、多角形を三角形分割する方法として「耳刈り取り法」のモジュールを利用します

単調多角形を三角形分割する手法(耳刈り取り法)
* [earcut-python](https://github.com/joshuaskelly/earcut-python)

In [13]:
from earcut import earcut
import pyproj
import open3d as o3d

class Building:
    """ bldg:Building"""

    def __init__(self, from_srid, to_srid):
        # 元の平面データ
        self.polygons = []

        # 平面データの変換した座標系のデータ
        self.vertices = []
        # 三角形の頂点情報
        self.triangles = []
        # 三角形のメッシュデータ
        self.triangle_meshes = []

        # 座標変換の定義をします： 変換元座標系 → 変換先座標系
        self.transformer = pyproj.Transformer.from_crs(f"epsg:{from_srid}", f"epsg:{to_srid}")

    def get_triangle_mesh(self):
        # 三角形のメッシュデータを返す
        return self.triangle_meshes

    def transform_coordinate(self, latitude, longitude, height):
        # 座標変換： 変換元座標系 → 変換先座標系
        xx, yy, zz = self.transformer.transform(latitude, longitude, height)
        return np.array([xx, yy, zz])

    def create_triangle_meshes(self, polygons):
        # 三角形のメッシュデータ作成
        for plist in polygons:
            # 平面データ座標変換
            vertices = [self.transform_coordinate(*x) for x in plist]
            # 単調多角形を三角形分割(耳刈り取り法)
            vertices_earcut = earcut(np.array(vertices).flatten(), dim=3)
            if len(vertices_earcut) > 0:
                # 三角形の頂点情報を設定
                # 三角形の頂点情報は、平面データ座標のインデックス番号
                vertices_length = len(self.vertices)
                self.vertices.extend(vertices)
                triangles = np.array(vertices_earcut).reshape((-1, 3))
                triangles_offset = triangles + vertices_length
                self.triangles.extend(triangles_offset)

        # 三角形のメッシュデータ作成
        triangle_meshes = o3d.geometry.TriangleMesh()
        triangle_meshes.vertices = o3d.utility.Vector3dVector(self.vertices)
        triangle_meshes.triangles = o3d.utility.Vector3iVector(self.triangles)

        # 三角形のメッシュの法線取得
        triangle_meshes.compute_vertex_normals()

        # データを保存
        self.triangle_meshes = triangle_meshes
        self.polygons = polygons


In [14]:
# LOD2のデータを取得
def lod2(tree, nmap, from_srid, to_srid):
  """ lod2 """
  obj_buildings = []

  # bldg:Building 建物データを検索
  buildings = tree.xpath('/core:CityModel/core:cityObjectMember/bldg:Building', namespaces=nsmap)

  # 建物データでループ
  for building in buildings:
    # 建物クラスを作成
    obj_building = Building(from_srid, to_srid)

    # bldg:GroundSurface, bldg:RoofSurface, bldg:RoofSurface
    # LOD2の屋根、壁、床の面データのパスを定義
    polygon_xpaths = ['bldg:boundedBy/bldg:GroundSurface/bldg:lod2MultiSurface/gml:MultiSurface/gml:surfaceMember/gml:Polygon',
                      'bldg:boundedBy/bldg:RoofSurface/bldg:lod2MultiSurface/gml:MultiSurface/gml:surfaceMember/gml:Polygon',
                      'bldg:boundedBy/bldg:WallSurface/bldg:lod2MultiSurface/gml:MultiSurface/gml:surfaceMember/gml:Polygon']

    # LOD2の屋根、壁、床の面データの位置情報を取得
    vals_list = []
    for polygon_xpath in polygon_xpaths:
      poslist_xpaths = building.xpath(polygon_xpath, namespaces=nsmap)
      for poslist_xpath in poslist_xpaths:
        vals = poslist_xpath.xpath('gml:exterior/gml:LinearRing/gml:posList', namespaces=nsmap)
        vals_list.extend(vals)

    # [X, Y, Z]の座標配列を作成
    polygons = [str2floats(v).reshape((-1, 3)) for v in vals_list]
    # 三角形のメッシュデータ作成
    obj_building.create_triangle_meshes(polygons)
    # 建物クラスデータを保存
    obj_buildings.append(obj_building)

  return obj_buildings

In [15]:
# PLYファイルに保存
def write_ply(output_filename, obj_buildings):
  # 保存用ファイル名を作成
  basedir = os.path.dirname(output_filename)
  basename_without_ext = os.path.splitext(os.path.basename(output_filename))
  basename = basename_without_ext[0]
  extname = basename_without_ext[1]

  # 建物クラスでループ
  for index, obj_building in enumerate(obj_buildings):
    # 三角形のメッシュデータを取得
    triangle_mesh = obj_building.get_triangle_mesh()
    # PLYファイルに保存
    pathname = os.path.join(basedir, f"{basename}_{index:02}{extname}")
    o3d.io.write_triangle_mesh(pathname, triangle_mesh, write_ascii=True)

In [16]:
# LOD2の建物データを作成
obj_buildings = lod2(tree, nsmap, from_srid, to_srid)
# PLYファイルに保存
output_filename = f"/content/drive/MyDrive/13.Colab Notebooks/dataset/citygml_out/{mesh_code}_{object_name}_{to_srid}_lod2.ply"
write_ply(output_filename, obj_buildings)

In [17]:
# Plotlyをインポート
import plotly.graph_objects as graph_objects

def show_ply(show_filename):
  # メッシュタイプ（PLYファイル）のデータを読み込み
  mesh = o3d.io.read_triangle_mesh(show_filename)

  # 法線を計算
  mesh.compute_vertex_normals()    # 頂点の法線を計算
  mesh.compute_triangle_normals()  # 三角形の法線を計算

  # 頂点と三角形のデータをnp.array形式に変換
  triangles = np.asarray(mesh.triangles)
  vertices = np.asarray(mesh.vertices)

  # Plotlyで表示。出力結果はマウスでインタラクティブに動かすことが可能です
  fig = graph_objects.Figure(
      data=[
          graph_objects.Mesh3d(
              x=vertices[:,0],   # 頂点のX座標
              y=vertices[:,1],   # 頂点のY座標
              z=vertices[:,2],   # 頂点のZ座標
              i=triangles[:,0],  # 三角形の1番目の座標
              j=triangles[:,1],  # 三角形の2番目の座標
              k=triangles[:,2],  # 三角形の3番目の座標
              facecolor= (0.5, 0.5, 0.5),  # 面の色を設定
              opacity=1.0)      # 表面の不透明度を設定
      ],
      layout=dict(
          scene=dict(
              xaxis=dict(visible=False), # X軸非表示
              yaxis=dict(visible=False), # Y軸非表示
              zaxis=dict(visible=False)  # Z軸非表示
          )
      )
  )
  fig.show()  # 表示

In [18]:
# PLYファイル表示(LOD2 - No.1の建物)
show_ply("//content/drive/MyDrive/13.Colab Notebooks/dataset/citygml_out/53392633_bldg_6677_lod2_01.ply")

In [ ]:
# PLYファイル表示(LOD2 - No.5の建物)
show_ply("/content/drive/MyDrive/13.Colab Notebooks/dataset/citygml_out/53392633_bldg_6677_lod2_05.ply")